In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.cosmology as Cosmology
from astropy import units as u 
from astropy import constants as const
from ipynb.fs.full.Analytic_neutrino_flux import *
import math
from IPython.display import display
import pandas as pd
import time
from astropy.coordinates import SkyCoord


### Used GOALS data

## Herschel IR luminosity data

In [2]:
Herschel = np.loadtxt("/Users/yarno/documents/PhD/GOALS/SplitLIR/goals_herschel_sample_list.txt")

Split LIR values: https://goals.ipac.caltech.edu/data_files/Lir_LirSurfaceDensity.dat -> IDs missing: 45, 46 , 78, 83, 110, 150, 157,  159 ,187, and 189 + 92 & 238 NaN values. This results in LIR values for 229 galaxies. Note the GOALS sample consists of 202 objects. See also the paper related to the analysis: https://ui.adsabs.harvard.edu/abs/2017ApJ...846...32D/abstract
 
Names corresponding to the iDS in the above file: https://goals.ipac.caltech.edu/data_files/goals_herschel_pacs_CII158_linecatalog_HIPEv13.dat

The redshift and luminosity distance values can be found in "/Users/yarno/documents/PhD/GOALS/goals_herschel_sample_list.txt"

In [3]:
LIR_split_array = np.loadtxt("/Users/yarno/documents/PhD/GOALS/SplitLIR/lir_split.txt", skiprows=0)
LIR_split = [i[1] for i in LIR_split_array]
LIR_split_unc = [i[2] for i in LIR_split_array]
ID = [int(i[0]) for i in LIR_split_array]

LIR_irg = [i[1] for i in LIR_split_array if i[1] < 1]
LIR_lirg = [i[1] for i in LIR_split_array if i[1] >= 1 and i[1] < 10]
LIR_ulirg = [i[1] for i in LIR_split_array if i[1] >= 10]

SigmaIR = [i[3] for i in LIR_split_array ]
SigmaIR_noNaN = []
for i in SigmaIR:
    if math.isnan(i) == False:
        SigmaIR_noNaN += [i]
    else: 
        SigmaIR_noNaN += [0]



## Individual  $\langle \alpha_{\mathrm{AGN}} \rangle$ distribution

In [4]:
fig3 = plt.figure(figsize=(10,9))

AGN_split_array = np.loadtxt("/Users/yarno/documents/PhD/GOALS/SplitLIR/agn_split.txt", skiprows=0)
agn_fracs_unc = [i[1] for i in AGN_split_array]
agn_fracs = [i[0] for i in AGN_split_array]
median_ang_frac = np.median(agn_fracs)


agn_fracs = np.array(agn_fracs)


<Figure size 720x648 with 0 Axes>

The AGN values can be found in Table 2 of this article: https://iopscience.iop.org/article/10.3847/1538-4357/aa81d7#apjaa81d7t1 . It is noted that the IDS 45, 46 , 78, 83, 110, 150, 157,  159 ,187 and 189 are already missing and therefore 92 & 238 had to be removed.


Note, that the mid-infrared and bolometric AGN fractions are both derived from the Spitzer low-res spectra, and are therefore representative of the projected physical area covered by the IRS short-low slit, centered on the nucleus. For more distant or point-like GOALS sources (D > ~ 50-100 Mpc), the AGN fractions will be representative of the values for the entire galaxy.  However, for more nearby sources, the true global mid-infrared and bolometric AGN fractions can be significantly smaller than those reported here. For example, the source with an average AGN fraction of one is NGC 1068, the most nearby (~16 Mpc) Seyfert II galaxy. The true global bolometric AGN fraction will therefore, most likely, be lower.

# Constructing a general dataframe

In [5]:
def SNr(LIR): # SN-LIR empirical callibration S. Matilla
    return 2.7e-12*LIR

def SFR(LIR,calib):
    if calib == "Murphy":
        return 3.15e-44*LIR*1e7*3.828e26
    if calib == "Yarno NK":
        return 4.934702e-44*LIR*1e7*3.828e26 
    if calib == "Yarno TH":
        return 1.537522e-44*LIR*1e7*3.828e26

def SNr_IMF(LIR,calib):
    if calib == "Murphy":
        return (1/86.3)*3.88e-44*LIR*1e7*3.828e26
    elif calib == "Yarno NK":
        return 5.428172e-46*LIR*1e7*3.828e26 
    elif calib == "Yarno TH":
        return 4.151310e-46*LIR*1e7*3.828e26


The callibration for the SFR is based on 33GHz continuum data for 56 nuclei and 62 extranuclear regions for star-forming galaxies covering a wide range of integrated properties, ISM conditions, morphological types, IR luminosity range, and and star-formation rates. The Median value for the ratio between the 33GHz star-formation rate and the IR luminosity are consistent for nuclear and extra-nuclear regions. This empirically determined coefficient is consistent with a theoretical relation given in Murpy et al (2011, SFR = 3.88e-44L$_{IR}$,https://ui.adsabs.harvard.edu/abs/2011ApJ...737...67M/abstract and https://ui.adsabs.harvard.edu/abs/2012ApJ...761...97M/abstract).

In [ ]:
a_file = open("/Users/yarno/documents/PhD/GOALS/SplitLIR/IDS.txt", "r")

list_of_lists = [] 
for line in a_file:
    stripped_line = line.strip()
    line_list = stripped_line.split()
    list_of_lists.append(line_list)
missing = [45, 46 , 78, 83,92, 110, 150, 157,  159 ,187, 189,238]

l = [[int(i[0]),i[1],i[6],i[7]] for i in list_of_lists if int(i[0]) not in missing]
l2 = [[i[3],i[4]] for i in Herschel if int(i[0]) not in missing]

dec_l_rad = np.array([i[2] for i in Herschel if int(i[0]) not in missing ])*0.0174532925

chars = [
    
          [
        
          ID[i],
          
          round(np.log10(LIR_split[i]*1e11),2), 
          
          round(LIR_split_unc[i],2), 
          
          agn_fracs[i], 
          
          agn_fracs_unc[i], 
          
          round(SFR((1-agn_fracs[i])*LIR_split[i]*1e11,"Yarno NK"),2),
                    
          round(SFR(LIR_split[i]*1e11,"Yarno NK"),2) ,
              
          SigmaIR[i]
          
         ]
         
          for i in range(len(ID)) if ID[i] != 92 and ID[i] != 238 
         ]

    
for i in range(len(chars)): 
    chars[i].insert(1,l[i][1])
    chars[i].insert(2,l[i][2])
    chars[i].insert(3,l[i][3])
    chars[i].insert(4,l2[i][0])
    chars[i].insert(5,l2[i][1])


In [ ]:
E = 1e3
nism = 1000 
R = 150
v= 500
pmax = 1e8
H = 150
df = pd.DataFrame({ 'Name' : [i[1] for i in l],
        
        'RA' : [round(SkyCoord(i[2],i[3]).ra.deg,2)  for i in l],
        
        'Dec' : [round(SkyCoord(i[2],i[3]).dec.deg,2)  for i in l],
                   
        'Redshift': [i[4] for i in chars],
        
        'D_L [Mpc]' : [ i[5] for i in chars],
        
        'log(LIR)' : [i[6] for i in chars],
        
        'LIR_unc x 1e11' : [i[7] for i in chars],
        
        'AGNbol' : [i[8] for i in chars],
        
        'AGNbol_unc' : [i[9] for i in chars],
        
        'SFR [M$_{\odot}$]' : [i[10] for i in chars ],
                           
        'un-corr SFR [M$_{\odot}$]':[i[11] for i in chars ] ,
                   
        'Supernova rate [yr$^{-1}$] ' : [round(SNr_IMF((1-i[8])*pow(10,i[6]), "Yarno NK"),2) for i in chars],
                   
        'un-corr Supernova rate [yr$^{-1}$] ' : [round(SNr_IMF(pow(10,i[6]), "Yarno NK"),2) for i in chars],
        
        'Flux(TeV) [TeV$^{-1}$ cm$^{-2}$ s$^{-1}$]': [round(Flux(1e3,R,v,nism,H,4,pmax,SNr_IMF((1-i[8])*pow(10,i[6]), "Yarno NK"),i[5]),13)*1e-3*1e12 for i in chars],
                                                                 
        'Flux(TeV) no AGN [GeV cm$^{-2}$ s$^{-1}$]': [round(Flux(1e3,R,v,nism,H,4,pmax,SNr_IMF(pow(10,i[6]), "Yarno NK"),i[5]),13)*1e-3*1e12 for i in chars],                   
        
                    
       }, index = [i[0] for i in chars])

pd.set_option('display.max_rows', 500)

display(df)


In [ ]:
print(df[["Name","RA","Dec", "D_L [Mpc]", "log(LIR)", "AGNbol",'Supernova rate [yr$^{-1}$] ','Flux(TeV) [TeV$^{-1}$ cm$^{-2}$ s$^{-1}$]']].to_latex(caption = "Table",index = False))